## Overview
In compile time programming, the compiler executes code at compile time. This result of the computation is available for the program at runtime. There will be no run time overhead, hence results in faster execution of the program. Compile time programming could be done in C using preprocessor(first stage of compilation) macro functions. We can give directives to the preprocessor, which start with a # symbol. For macro functions we use the #define directive, which defines a symbol and the preprocessor will replace that symbol by what ever you define it as. You can provide arguments too, the arguments will be substituted.
```
#define MAX(x, y) ((x) > (y) ? (x) : (y))

MAX(++a, b); //Will be replaced with ((++a) > (b) ? (++a) : (b)) during preprocessing
             //a may be increamented twice(if a++ > b)
```
There are a few problems with this, there is no type safety as it preprocessing, we should always put brakets around the arguments due to the operator precedence rules and if any of the arguments have side effects then they may be evaluated more than once(like a++ above).

C++ has templates, these were intended to support generic programming. Meaning with templates we can write code to work with a generic type, then the compiler will generate code for the provided type. 

Later on it was discovered that templates provide a turing complete programming language(we can solve any problem using templates in theory).
* We can use template parameters to represent state variables
* We can use recursive instantiation to simulate loops
* We can use template specialization to implement control flow
* We can also use integer operations to calculate results(to do math)

Template meta programming can be used for
* Compile-time programming as stated above
* It can be used for the original intended purpose, writing generic code to avoid duplication(e.g STL)
* We can use it to make decisions at compile time for efficiency/portability(write code to generate code based on harware)
* Develop a domian-specific language on top of C++
* Express complex software patterns and concepts

Template classes have a convention, members that represent numeric results are named 'value' and members that represent types are named 'type'. C++ has type traits to get information about the properties of types. For example if we want to know if the template type is a arthematic type then we can use is_arithmatic\<T\>.
```
std::cout << std::boolalpha;
std::cout << std::is_arithmatic<int>::value << std::endl;           //true
std::cout << std::is_floating_point<int>::value << std::endl;       //flase
std::cout << std::is_class<A>::value << std::endl;                  //true if A is a user defined class
std::cout << std::is_pointer<const char*>::value << std::endl;      //true
```

Until C++ 11 the only real option for compile time programming is templates, which is very verbose, complicated with no debug support. From C++ 11 we have constexpr, which is like normal C++ code executed by the compiler at compile time, hence it is easy and can be debugged. 

## Constant Expressions
A constant expression has a value that is evaluaed at compile time and which cannot change. Literals, values computed from literals and values computed from other constant expressions can be constant expressions. A variable can be a constant expression(use const keyword), it cannot be subsequently modified.
```
const int i{42}, j{99};

i + j; //result of i+j is again a constant expression

int a[i+j]; //ok, as i+j is a constant expression, this will give a compiler error if i or j is not const 
```
We can also give a value to a const variable that is only known at runtime, this is not compile time programming.
```
int x;
std::cin >> x;
const int y{x};
```
From C++ 11 there is now a constexpr keyword which can be applied to variables and functions.
```
constexpr int i{3}; //i is evaluated at compile time and connot be modified
```

## Constexpr Functions
This means that the function arguments and return value are constant expressions. These functions are executed at compile time, using a compile time interpreter that support most of C++. 
```
constexpr double miles_to_km(double miles)
{
    return miles * 1.602;
}

const double dist1 = miles_to_km(40); // or constexpr double dist1
```
constexpr functions must be pure, meaning it cannot modify its arguments and connot modify global/static variables. constexpr functions are inline, the function calls are emitted with the function code. Inline also means it can have multiple definitions, we can define it directly header and include it where ever required(like inline static member variables). A constexpr function can be called with arguments that are not constant expressions, it this case the function will be evaluated at runtime like normal C++ function. This will allow as not to write 2 identical versions of the same function. The body of a constexpr function should not contain code which causes a action at runtime, like new/delete(we cannot use things like std::string and std::vector), calling virtual functions, throwing exceptions etc. Member functions can also be constexpr, member variables can also be constexpr provided they are static.

## Template Classes
Template classes work very much like other classes with templated parameters. A class or a template class can have a member template, the member template can have different template parameters from the class, a member template cannot be virtual. Lambda expressions with auto parameters/return type are implemented as classes with a member template.
```
auto lambda = [](auto x, auto y)
{
    return x + y;
};

//Implemented as(compiler generated code)
class functor
{
public:
    template <typename T>
    T operator (T x, T y)
    {
        return x + y;
    }
};

lambda(2, 5); //compiler will regenerate int type member template and instantiate
```
Example of a member template which has different template parameters from the class
```
template <typename T>
class comparer
{
public:
    comparer(const T& t1, const T& t2) : m_t1(t1), m_t2(t2)
    {
    }
    
    //Member template with paramter func
    template <typename func>
    bool compare(func f)
    {
        return f(m_t1, m_t2);
    }

private:
    T m_t1, T m_t2;
};

int main()
{
    int x{1}, y{2};
    comparer<int> c(x, y);
    
    //Pass a callable object as argument
    bool b = c.compare([](int i1, int i2) { return i1 < i2; } );
    std::cout << boolalpha << b << std::endl;
}
```
C++ has concepts, concepts allow us to express constraints on template parameters. We can specify these requirements as part of the template definition. For example in the above code we can state that func has to be a callable object, and that class members have to be comparable.

## Template Specialization
Templates are generic, we get the same behaviour for every type that we instantiate it with. But sometimes we wish to have different behaviour for some types, when we want to handle some types differently, or when the code does not behave correctly for some types, or when we want to optimize the code for some types. We can do this by using "template specialization".
```
//Generic vector class
template <typename T>
class vector
{
    ...          //Implement using an array of T
};

//Secialization of the vector class for bool type
template <>
class vector<bool>
{
    ...         //Implement using a bitmap
}
```
The specialization should follow immediately after the generic template, else it will not compile. The compiler will choose the most specific alternative which matches.
```
vector<bool> vec;   //Chooses the bool alternative 
```
Example of template function specialization for containers
```
template <typename T>
void reverse(T& container)
{
    std::reverse(std::begin(container), std::end(container));
}

//As list itself is a class template, we need to provide parameter for list's element type, 
//as we want this function to be used for any list type
template <typename elem>
void reverse(std::list<elem>& container)
{
    container.reverse();
}

int main()
{
    vector<int> vec{3, 1, 4, 1, 5, 9};
    reverse(vec);  //Generic version will be called
    
    list<int> lis{3, 1, 4, 1, 5, 9};
    reverse(lis);  //List specialized version will be called
}
```
For the above we have to write different type for the specialization of each type. But if the types are related in someway then we can write a partial specialization for the related types. For example we can write partial specialization for pointer types.
```
//Generic vector class
template <typename T>
class vector
{
    ...
};

// Partial specialization for pointer elements
template <typename T>
class vector<T*>
{
    ...
};

vector<int> vec;     //Intantiates generic vector class
vector<int*> vec;    //Intantiates partial specialization of vector class
```

## Entern Templates
When we have a template we normally put the full template definition in a header. If it is a function template the full function body goes into the header, for class template the full class definition goes into the header(no .cpp file). The reason for this is when we use the template in a source code file, the compiler should see the full definition of the template to generate and intantiate it for the specified type. If we have the same intantiation in multiple files, then the compiler will generate same template in every single file.
```
void func(arg<T>)
{
    ...
}

//File_001.cpp
func(this_string);   //Generates func(string) in File_001.cpp and intantiates
...
//File_269.cpp
func(this_string);   //Generates func(string) in File_269.cpp and intantiates
```
This is known as the template bloat. However the linker will remove these duplicate definitions from the executable, but the bloat increases the object files size and compilation time. The traditional way of solving this is to keep only the template declartion in the header, definition in the corresponding .cpp file and manually intantiate the specific templates in the source code files. This is not clean, hence extern templates where introduced.

The extern keyword is inherited from C, used to make global variable accessible accross different source files. In modern C++ we can also use the extern keyword with templates. We declare the template in header, followed by all the specific template declarations with the extern keyword. The template .cpp file will have the template difinition, followed by all the specific template declarations without the extern keyword(actual declaration here, as extern indicates that it is declared else where). This will make sure that there is only instatiation in the entire program for a type.
```
//Header
template <typename T>
void func(arg<T>);

extern template void func(string);

//.cpp
template <typename T>
void func(arg<T>)
{
    ...
}

template void func(string);

//In File_001.cpp to File_269.cpp
func(this_string);    //Specific template will not be generated as it is declared else where
```

## Variadic Templates
Variadic functions can take any number of arguments. Example, the printf function can take any number of arguments. This is not type safe as all the argument processing here is done at runtime. Modern C++ has variadic template functions, which are done at compile time and they are type safe.
```
//Template function with variable number of arguments
template <typename... Args> //Args is a list of tempalte parameter types
void func(Args... args); //args is a list of arguments whose types match the template arguments

//Equivalent to template <typename T>
//func(T t);
func("hello"s);   //Intantiated as func(string);

//Equivalent to template <typename T, typename U, typename V>
//func(T t, U u, V v);
func(42, 0.0, "text"s);   //Intantiated as func(int, double, string);
```
These lists with ... are known as parameter packs. Above template function has 2 paramter packs, one that represents template parameters and one that represents the function arguments. The compiler will deduce the number of arguments and thier types from the calls and generates the code(template) for instantiation. The parameter packs are only available at compile time. We can do 3 things with a paramter pack, use sizeof...() to get the number of elements, use make_tuple() to store elements in a tuple(this also makes elements available at runtime) and we can iterate over the elements using template recursion.
```
template <typename... Args>
void func(Args... args)
{
    //At compile time this will be replaced with a sizeof([list])
    std::cout << sizeof...(args) << std::endl;
    
    auto arg_tuple = make_tuple(args...);
    auto first = get<0>(arg_tuple);
    std::cout << first << std::endl;
}
```
Normal runtime recursion is based on changing the argument's values for each successive call and having a exit condition based on the argument's values. Template recursion is based on changing the number of arguments.
```
template <typename T, typename U, typename V>
void func(T t, U u, V v)
{
    //Do something with t
    func(u, v);
}

template <typename U, typename V>
void func(U u, V v)
{
    //Do something with u
    func(v);
}

template <typename V>
void func(V v)
{
    //Do something with v
    //End of recursion
}
```
In the above example if you imagine the number of arguments as list elements then each function above is processing one element of the list. This is like iterating over the elements of a list. With variadic templates we can iterate over any number of elements.
```
//Function with single argument
template <typename T>
void func(T t)
{
    //End of recursion
}

//When you call this function with 4 parameters, first parameter will be t and the remianing 3 will be in args
template <typename T, typename.. Args>
func(T t, Args... args)
{
    //Process t here
    
    //The compiler will replace args.. with the list of arguments
    func(args...);  //Recursive call, pass the remainder of the list
}

int main()
{
    int i{42};
    double d{0.0};
    std::string s{"text"};
    
    func(i, d, s);
}
```
We need the function with single argument to the end the recursion. This should be above the variadic function.

## More Template Features
std::assert() macro is inherited from C. It checks the argument at runtime, if the argument is 0 it calls std::abort(), which will terminate the program immediately.
```
std::assert(x == 42);     //If x value is not 42, the program will be terminated
```
There is a runtime overhead for doing these checks. Also these checks should not be present in the production code, these checks can be disabled by #define NDEBUG macro(std::assert() does nothing). 
In C++ we have a compile time assert, std::static_assert(). It takes a constant bool expression and a string literal. If the bool expression evaluates to false, the compiler will stop compilation and throw error message with the given string literal in it.
```
std::static_assert(sizeof(int*) == 8, "This program requires a 64-bit compiler");

//Mainly useful in template meta programming
std::static_assert(std::is_copy_constructible<T>::value, "Argument must be copyable");
```
We can write template class with default parameters(types). The same applies to template functions too.
```
template <typename T = int>
class number
{
    T value;  //value is int type by default
};

number<long double> high_precision{1.99999};   //long double object

number<> default_number{1.99999}; //int object(default)
```

## Library Defined Operators
C++ defined some generic operator classes in \<functional\>, e.g less\<Type\> calls the '<' operator of Type. They cover all arthematic, relational, logical and bitwise operators. These will just call the corresponding operator of the Type. These are functor classes, so we need to create an object and then call its () operator - less()(t1, t2). We can use these functions for writing generic functions with default comparisions, similar to std::sort(will use less than operator for default comparision if the predicate is not provided).
```
template <typename T, typename Func = less<T>>
bool compare(const T& t1, const T& t2, Func func = Func())
{
    return func(t1, t2);
}

compare(x, y, [](int i1, int i2){return i1 < i2;});    //Provides a callable object, hence uses it

compare(x, y);   //Callable object not provided, hence uses the library's less<T> operator
```

## Constexpr If Statement
Also known as if constexpr or static if, allows conditions to be evaluated at compile-time.
```
if constexpr(a < b)    //If a and b are constant expressions this will be evaluated at compile time
...                    //if a < b, if code will be compiled into the program, else the else code will be compiled
else
...
```
Therefore we can use constexpr if to do conditional compilation. The traditional preprocessor directives #if and #ifdef to conditionally include or exclude code from compilation had a number of drawbacks. The directives are processed by the preprocessor, which does not understand C++, types and may have side effects.
```
//If the arg type is std::string if part of the code is compiled, if arg is any other type else part of the code is compiled
template <typename T>
string get_string(const T& arg)
{
    if consexpr(std::is_same<std::string, T>::value)    //Replaced by "true" if T is a std::string, else false
    {
        return arg;                                     //std::to_string() does not support std::string argument
    }
    else
    {
        std::to_string(arg);
    }
}
```
Before C++ had constexpr if, people also solved this problem with template specialization. In above case a generic get_string method and a specialized one for std::string. The advantage with constexpr is that it is a normal C++ code run at compile time. 

## Example
Compile time finonacci using templates.
```
//General case (N > 1)
template <int N>
int fibonacci()
{
    return fibonacci<N-1>() + fibonacci<N-2>();
}

//Specialized for N = 1
template <>
int fibonacci<1>()
{
    return 1;
}

//Specialized for N = 0
template <>
int fibonacci<0>()
{
    return 0;
}

int main()
{
    constexpr int n{10};
    cout << fibonacci<n>() << std::endl;
}
```
Compile time finonacci using constexpr if is much simpler.
```
template <int N>
constexpr int fibonacci()
{
    if consexpr (N > 1)
    {
        return fibonacci<N-1>() + fibonacci<N-2>();
    }
    return N;
}
```
The variadic template used to iterate in a loop at compile time(last example in variadic templates) also becomes lot simpler with constexpr if.
```
template<typename T, typename... Args>
void func(T t, Args... args)
{
    size_t n_args = sizeof...(args) + 1;
    if constexpr(sizeof...(args) > 0)
    {
        func(args...);
    }
}
```

## Decltype
Decltype gives the type of its argument at compile type. Some compilers had a non standard extension 'typeof' before decltype was introduced. decltype does not evaluate its argument or return any code, the compiler will just replace decltype with the type of the argument.
```
decltype(1 + 2) y; //int y, decltype(1 + 2) is replaced with int since 1 + 2 is of type int

int x;
decltype(x) z;   //int z
```
Unlike auto decltype gives same type of a existing object or expression, it will also retain const, reference, etc.
```
const int cx;          //The type of cx is const int
auto y = cs;           //The type of y is int, auto will drop const, reference, etc.
decltype(cx) cz;       //The type of cz is const int
```
decltype with argument of lvalue expression, then the resultant type will be a lvalue reference to the deduced type of the expression.
```
int x;
int y;
decltype(x+y)       //Gives int&, since x + y is a lvalue expression
decltype((x))       //Gives int&, since brackets (x) makes it a expression 
```
If we have a xvalue(an expiring value, temporary value), then the compiler will deduce the type and replace decltype with rvalue reference.
```
decltype(test())    //Gives test&&
```
decltype is mainly used for compile time return types.
```
template <typename T, typename U>
auto add(T t, U u) -> decltype(t + u)    //Compiler will replace auto with the type deduced from decltype(t + u)
{
    return t + u;
}
```